<a href="https://colab.research.google.com/github/mudit458/face-recognition/blob/main/Face_RecognitionColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


%cd /content/gdrive/My Drive/

!rm -rf dataset
!rm -rf lfw.tgz

!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz



Mounted at /content/gdrive
/content/gdrive/My Drive
--2021-05-22 10:32:03--  http://vis-www.cs.umass.edu/lfw/lfw.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180566744 (172M) [application/x-gzip]
Saving to: ‘lfw.tgz’

lfw.tgz             100%[===================>] 172.20M  51.6MB/s    in 3.4s    

2021-05-22 10:32:07 (50.1 MB/s) - ‘lfw.tgz’ saved [180566744/180566744]



In [ ]:
!rm -rf lfw

!tar zxvf lfw.tgz

!rm -rf lfw.tgz

Streaming output truncated to the last 5000 lines.
lfw/Andy_Benes/Andy_Benes_0001.jpg
lfw/Chuck_Finley/
lfw/Chuck_Finley/Chuck_Finley_0001.jpg
lfw/Mike_Matheny/
lfw/Mike_Matheny/Mike_Matheny_0001.jpg
lfw/Wang_Hailan/
lfw/Wang_Hailan/Wang_Hailan_0001.jpg
lfw/Eric_Vigouroux/
lfw/Eric_Vigouroux/Eric_Vigouroux_0001.jpg
lfw/Leonardo_DiCaprio/
lfw/Leonardo_DiCaprio/Leonardo_DiCaprio_0001.jpg
lfw/Leonardo_DiCaprio/Leonardo_DiCaprio_0002.jpg
lfw/Leonardo_DiCaprio/Leonardo_DiCaprio_0003.jpg
lfw/Leonardo_DiCaprio/Leonardo_DiCaprio_0004.jpg
lfw/Leonardo_DiCaprio/Leonardo_DiCaprio_0005.jpg
lfw/Leonardo_DiCaprio/Leonardo_DiCaprio_0006.jpg
lfw/Leonardo_DiCaprio/Leonardo_DiCaprio_0007.jpg
lfw/Leonardo_DiCaprio/Leonardo_DiCaprio_0008.jpg
lfw/Leonardo_DiCaprio/Leonardo_DiCaprio_0009.jpg
lfw/Warren_Beatty/
lfw/Warren_Beatty/Warren_Beatty_0001.jpg
lfw/Warren_Beatty/Warren_Beatty_0002.jpg
lfw/Janusz_Kaminski/
lfw/Janusz_Kaminski/Janusz_Kaminski_0001.jpg
lfw/Martin_Landau/
lfw/Martin_Landau/Martin_Landau_0

# Dataset Added !!!

In [ ]:
import csv
from shutil import rmtree
from time import sleep

import cv2
import numpy as np
import math
import os
import pandas as pd
import shutil
import matplotlib.pyplot as plt

FACE_CASCADE = cv2.CascadeClassifier('haarcascade/haarcascade_frontalface_default.xml')
EYE_CASCADE = cv2.CascadeClassifier("haarcascade/haarcascade_eye.xml")



# Clean Data


- Rename Text to INT
- Remove all Labels with too few images [20]
- Remove too many images to maximum 50

In [ ]:
DATA_PATH = "dataset"

ORIGINAL_DATA_PATH = "lfw"

if os.path.isdir(DATA_PATH):
  shutil.rmtree(DATA_PATH)

path = ORIGINAL_DATA_PATH
cnt = 0
for root, directory, filenames in os.walk(path):
    if len(filenames) <= 20:
        # don't use this
        continue
    cnt += 1
    img_cnt = 0
    for filename in filenames:
        img_path = os.path.join(root, filename)
        img = cv2.imread(img_path)
        dest_path = os.path.join(DATA_PATH, str(cnt))
        if not os.path.isdir(dest_path):
            os.makedirs(dest_path)
        cv2.imwrite(os.path.join(dest_path, filename), img)
        img_cnt += 1
        if img_cnt >= 50:  # maximum 30 images
            break

# Dataset Visualization

In [ ]:
cleaned_data_count = {}
original_data_count = {}

pos = {}

# Cleaned Data
for root, directory, filenames in os.walk("dataset"):
    label = os.path.basename(root)
    if label not in pos:
        pos[label] = len(pos) + 1
    label = pos[label]
    cleaned_data_count[label] = len(filenames)

pos = {}
# Original Data
for root, directory, filenames in os.walk(ORIGINAL_DATA_PATH):
    label = os.path.basename(root)
    if label not in pos:
        pos[label] = len(pos) + 1
    label = pos[label]

    original_data_count[label] = len(filenames)

clean_pts = sorted(cleaned_data_count.items())

original_pts = sorted(original_data_count.items())
xs = [item[0] for item in original_pts]
ys = [item[1] for item in original_pts]
plt.plot(xs, ys)
plt.title('Original Data Distribution')
plt.show()


plt.bar([item[0] for item in clean_pts], [item[1] for item in clean_pts])
plt.title('Cleaned Data Distribution')
plt.show()

# Load all Images

In [ ]:
DataImages = []
ImageLabels = []

for root, directory, filenames in os.walk(DATA_PATH):
    for filename in filenames:
        img_path = os.path.join(root, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        DataImages.append(img)
        label = os.path.basename(root)
        ImageLabels.append(int(label))



In [ ]:
# Genereate Samples

from random import sample
samples = sample(list(zip(DataImages, ImageLabels)), 3)
print(samples[:2])


In [ ]:
''' show sample images with labels'''
def show_samples(samples):
    for sample in samples:
        img, label = sample
        imgplot = plt.imshow(img)
        plt.title(label)
        plt.show()

show_samples(samples)

# PREPROCESS FUNCTIONS


## Align Image

In [ ]:
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    resultImage = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return resultImage



def straightenFace(input_img, with_marks = False):
    eye_1, eye_2 = 0, 0
    face_img = input_img.copy()

    # accepts b/w  face cropped image
    original = face_img.copy()

    eyes = EYE_CASCADE.detectMultiScale(face_img)

    if len(eyes) < 2:
        return original

    eyes = sorted(eyes, key=lambda x: - x[2] * x[3])
    index = 0
    for (eye_x, eye_y, eye_w, eye_h) in eyes:
        if index == 0:
            eye_1 = (eye_x, eye_y, eye_w, eye_h)
        elif index == 1:
            eye_2 = (eye_x, eye_y, eye_w, eye_h)

        index += 1
        if index >= 2:
            break

    # set left and right eyes
    if eye_1[0] < eye_2[0]:
        left_eye, right_eye = eye_1, eye_2
    else:
        right_eye, left_eye = eye_1, eye_2
    left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + left_eye[3] / 2))
    left_eye_x = left_eye_center[0]
    left_eye_y = left_eye_center[1]

    right_eye_center = (int(right_eye[0] + (right_eye[2] / 2)), int(right_eye[1] + (right_eye[3] / 2)))
    right_eye_x = right_eye_center[0]
    right_eye_y = right_eye_center[1]

    cv2.circle(face_img, left_eye_center, 2, (255, 0, 0), 2)
    cv2.circle(face_img, right_eye_center, 2, (255, 0, 0), 2)
    cv2.line(face_img, right_eye_center, left_eye_center, (67, 67, 67), 2)

    if left_eye_y < right_eye_y:
        point_3rd = (right_eye_x, left_eye_y)
        direction = -1  # rotate same direction to clock
    else:
        point_3rd = (left_eye_x, right_eye_y)
        direction = 1  # rotate inverse direction of clock

    cv2.circle(face_img, point_3rd, 2, (255, 0, 0), 2)

    cv2.line(face_img, right_eye_center, left_eye_center, (67, 67, 67), 2)
    cv2.line(face_img, left_eye_center, point_3rd, (67, 67, 67), 2)
    cv2.line(face_img, right_eye_center, point_3rd, (67, 67, 67), 2)

    # cv2.imshow("original", face_img)
    # cv2.waitKey(0)
    def euclidean_distance(point_a, point_b):
        x1 = point_a[0]
        y1 = point_a[1]
        x2 = point_b[0]
        y2 = point_b[1]
        return math.sqrt(((x2 - x1) * (x2 - x1)) + ((y2 - y1) * (y2 - y1)))
    
    a = euclidean_distance(left_eye_center, point_3rd)
    b = euclidean_distance(right_eye_center, left_eye_center)
    c = euclidean_distance(right_eye_center, point_3rd)
    
    if (b == 0 or c == 0):
        return original

    cos_a = (b * b + c * c - a * a) / (2 * b * c)

    angle = np.arccos(cos_a)

    angle = (angle * 180) / math.pi

    if direction == -1:
        angle = 90 - angle
    else:
        angle = -angle

    img = rotate_image(original, angle)

    if with_marks:
        return cv2.hconcat([face_img, img])
    return img

image = DataImages[0]
results = []
for img, label in samples:
    st = straightenFace(img, True)
    if st is not None:
        results.append((cv2.hconcat([img, st]), label)) 
# results = [(cv2.hconcat([img, straightenFace(img, True)]), label) for img, label in samples]
show_samples(results)

*italicized text*## Crop Faces in image

In [ ]:
def show_image(img, label=None):
        imgplot = plt.imshow(img)
        if label is not None:
            plt.title(label)
        plt.show()

In [ ]:
def cropped_faces_in_image(img, sz = 0, show = False):

    faces = FACE_CASCADE.detectMultiScale(img, 1.3, 5)

    image = img.copy()
    res = []
    for face in faces:
        face_x, face_y, face_w, face_h = face
        cv2.rectangle(image, (face_x, face_y), (face_x + face_w, face_y + face_h), (0, 255, 0), 3)
        if sz == 1:
            face_x, face_y, face_w, face_h = face

            dx = int(face_w * .3)
            dy = int(face_h * .3)
            starty = face_y-dy
            if starty < 0:
                starty = 0
            endy = face_y + face_h + dy
            if endy > len(img):
                endy = len(img)

            startx = face_x-dx
            if startx < 0:
                startx = 0
            endx = face_x+face_w + dx
            if endx > len(img[0]):
                endx = len(img[0])
            cut_face = img[starty:endy, startx:endx]
        elif sz == -1:
            face_x, face_y, face_w, face_h = face

            dx = -int(face_w * .1)
            dy = -int(face_h * .07)
            starty = face_y-dy
            endy = face_y + face_h + dy

            startx = face_x-dx
            endx = face_x+face_w + dx


            cut_face = img[starty:endy, startx:endx]
        else:
            x, y, w, h = face
            
            cut_face = img[y:y + h, x:x + w]
        if show:
            show_image(image)
        res.append(cut_face)
    return res

img = DataImages[0]

# show_image(img, "NO CROP")
cropped_faces = cropped_faces_in_image(img, 0, show=True)
for cropped_face in cropped_faces:
    show_image(cropped_face, "ORIGINAL CROP")


small_cropped_faces = cropped_faces_in_image(img, -1)

for cropped_face in small_cropped_faces:
    show_image(cropped_face, "SMALL CROP")

big_cropped_faces = cropped_faces_in_image(img, 1)

for cropped_face in big_cropped_faces:
    show_image(cropped_face, "BIG CROP")

In [ ]:
def enhance_image(img):
    enhanced = cv2.equalizeHist(img)
    return enhanced


img = DataImages[0]
res = enhance_image(img)
plt.imshow(res)
plt.show()



# Enhance all images
for i in range(len(DataImages)):
    DataImages[i] = enhance_image(DataImages[i])

In [ ]:
def get_all_faces(img, one=False):
    # find all faces

    cropped_faces = cropped_faces_in_image(img, sz=1)


    if one and len(cropped_faces):
        # sort by area to get biggest face
        cropped_faces.sort(key=lambda x: - len(x) * len(x[0]))
        cropped_faces = cropped_faces[:1]


    # align these faces
    res = []
    
    for face in cropped_faces:
        st_face = straightenFace(face)
        if st_face is None:
            continue
        # close crop
        close_crop = cropped_faces_in_image(st_face, sz = -1)
        if len(close_crop):

            close_crop.sort(key=lambda x: -len(x) * len(x[0]))
            # resize face to 250, 250
            img = cv2.resize(close_crop[0], (250, 250)) # TODO
            res.append(img)

        # print(close_crop)

        # res.append(close_crop)
    return res


res = []
for image, label in samples[:3]:
    faces = get_all_faces(image, one=True)
    for face in faces:
        res.append(face)
for img in res:
    show_image(img)

In [ ]:
imgs = []
labels = []
for img, lab in zip(DataImages, ImageLabels):
    # try:
    faces = get_all_faces(img, one = True)
    # except:
    #     show_image(img)
    # since training data has only one face per image we will use the first image
    if len(faces):
        imgs.append(faces[0])
        labels.append(lab)
    # for f in faces:
    #     imgs.append(f)
    #     labels.append(lab)

DataImages = imgs
ImageLabels = labels

# Train Test Split

In [ ]:
msk = np.random.rand(len(DataImages)) < 0.8
trainX = []
trainY = []

testX = []
testY = []
for i in range(len(DataImages)):
    if msk[i]:
        trainX.append(DataImages[i])
        trainY.append(ImageLabels[i])
    else:
        testX.append(DataImages[i])
        testY.append(ImageLabels[i])

In [ ]:
trainX[:5]
# trainY[:5]

In [ ]:
testX[:3]
for face in testX[:3]:
    plt.imshow(face)
    plt.show()

# Eigen Face Recognizer

In [ ]:
recognizer = cv2.face.EigenFaceRecognizer_create()
import time, datetime

start = datetime.datetime.now()
recognizer.train(trainX, np.array(trainY))
end = datetime.datetime.now()

diff = (end - start)

diff

### On training data

In [ ]:
y_pred = []
for x in trainX:
    predictLabel, distance = recognizer.predict(x)
    y_pred.append(predictLabel)

# find accuracy
from sklearn.metrics import accuracy_score
accuracy_score(trainY, y_pred)

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
cf_mat = confusion_matrix(trainY, y_pred)

df_cm = pd.DataFrame(cf_mat)
plt.figure(figsize = (20,14))
import seaborn as sn
sn.heatmap(df_cm, annot=True)


### On new data

In [ ]:
y_pred = []
for x in testX:
    predictLabel, distance = recognizer.predict(x)
    y_pred.append(predictLabel)

# find accuracy
from sklearn.metrics import accuracy_score
accuracy_score(testY, y_pred)

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
cf_mat = confusion_matrix(y_true, y_pred)

df_cm = pd.DataFrame(cf_mat)
plt.figure(figsize = (20,14))
import seaborn as sn
sn.heatmap(df_cm, annot=True)


# Fisher Face Recognizer

In [ ]:
recognizer = cv2.face.FisherFaceRecognizer_create()
import time, datetime

start = datetime.datetime.now()
recognizer.train(trainX, np.array(trainY))
end = datetime.datetime.now()

diff = (end - start)

diff

### On training data

In [ ]:
y_pred = []
for x in trainX:
    predictLabel, distance = recognizer.predict(x)
    y_pred.append(predictLabel)

# find accuracy
from sklearn.metrics import accuracy_score
accuracy_score(trainY, y_pred)

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
cf_mat = confusion_matrix(trainY, y_pred)

df_cm = pd.DataFrame(cf_mat)
plt.figure(figsize = (20,14))
import seaborn as sn
sn.heatmap(df_cm, annot=True)


### On new data

In [ ]:
y_pred = []
for x in testX:
    predictLabel, distance = recognizer.predict(x)
    y_pred.append(predictLabel)

# find accuracy
from sklearn.metrics import accuracy_score
accuracy_score(testY, y_pred)

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
cf_mat = confusion_matrix(y_true, y_pred)

df_cm = pd.DataFrame(cf_mat)
plt.figure(figsize = (20,14))
import seaborn as sn
sn.heatmap(df_cm, annot=True)


# LBPH Recognizer

In [ ]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
import time, datetime

start = datetime.datetime.now()
recognizer.train(trainX, np.array(trainY))
end = datetime.datetime.now()

diff = (end - start)

diff

### On training data

In [ ]:
y_pred = []
for x in trainX:
    predictLabel, distance = recognizer.predict(x)
    y_pred.append(predictLabel)

# find accuracy
from sklearn.metrics import accuracy_score
accuracy_score(trainY, y_pred)

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
cf_mat = confusion_matrix(trainY, y_pred)

df_cm = pd.DataFrame(cf_mat)
plt.figure(figsize = (20,14))
import seaborn as sn
sn.heatmap(df_cm, annot=True)


### On new data

In [ ]:
y_pred = []
for x in testX:
    predictLabel, distance = recognizer.predict(x)
    y_pred.append(predictLabel)

# find accuracy
from sklearn.metrics import accuracy_score
accuracy_score(testY, y_pred)

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
cf_mat = confusion_matrix(y_true, y_pred)

df_cm = pd.DataFrame(cf_mat)
plt.figure(figsize = (20,14))
import seaborn as sn
sn.heatmap(df_cm, annot=True)
